<a href="https://colab.research.google.com/github/genaiconference/Agentic_RAG_Workshop/blob/main/01_Handling_Multi_Modal_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Document and Processing using Azure Document Intelligence

This notebook demonstrates how to:
- process a document using Azure Document Intelligence,
- extract information
- process images within the document & Interpret using llm
- Insert the image interpretation in place of the image

## Setup and Installations
Install necessary libraries for document processing, data handling, and interacting with Azure Document Intelligence and OpenAI.

In [1]:
!pip install -r requirements.txt

## Load Environment Variables and Initialize Clients
Load environment variables containing API keys and endpoint information, and initialize the Azure Document Intelligence and OpenAI clients.

In [7]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
import os

load_dotenv()

doc_intelligence_endpoint=os.getenv("doc_intelligence_endpoint")
doc_intelligence_key=os.getenv("doc_intelligence_key")


llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-4.1",
    temperature=0,
)

## Define Paths and Load Document

In [3]:
import os
import pickle
import di_utils
from IPython.display import Markdown

# --- Configuration ---
INPUT_FILE_PATH1 = "Leave Policy.pdf"
INPUT_FILE_PATH2 = "Insurance Policy.pdf"
PICKLE_DIR = os.path.join(os.getcwd(), "DI_output")
IMG_SAVE_DIR = os.path.join(PICKLE_DIR, "images")

# --- Ensure output directories exist ---
os.makedirs(PICKLE_DIR, exist_ok=True)
os.makedirs(IMG_SAVE_DIR, exist_ok=True)

## Run Document Intelligence

In [4]:
def run_document_intelligence(INPUT_FILE_PATH):
  """
  Analyzes a document using Azure Document Intelligence, extracts information,
  processes images within the document, and integrates image descriptions
  into the document's text representation.

  Returns:
      tuple: A tuple containing:
          - md_result: The markdown result object from Azure Document Intelligence layout analysis.
          - result_with_image_descp: A string containing the document's markdown text with image descriptions inserted.
  """
  # Step 1: Analyze Document Layout
  md_result = di_utils.analyze_document_layout(INPUT_FILE_PATH, doc_intelligence_endpoint, doc_intelligence_key)

  # Step 2: Build Span Map
  span_map = di_utils.extract_span_map(md_result)

  # Step 3: Process Images and Integrate with Text
  image_chunks = di_utils.process_images(INPUT_FILE_PATH, IMG_SAVE_DIR, md_result, llm)
  result_with_image_descp = di_utils.insert_figures_into_full_text(span_map, md_result, image_chunks)

  return md_result, result_with_image_descp

### Processing Leave Policy Document

In [5]:
md_result, result_with_image_descp = run_document_intelligence(INPUT_FILE_PATH1)

# Save results to a single pickle file as a dictionary
results_dict = {
    "md_result": md_result,
    "result_with_image_descp": result_with_image_descp
}

with open(os.path.join(PICKLE_DIR, "leave_policy_results.pkl"), "wb") as f:
    pickle.dump(results_dict, f)

Markdown(result_with_image_descp)

# LEAVE POLICY


## Objective

To provide guidelines to avail leave and to specify the leave procedures at Novartis
Hyderabad.


## Applicability

The policy is applicable to all associates on a permanent employment contract with the
Company.


## Types of Leave


<table>
<tr>
<th>Leave</th>
<th>Entitlement</th>
<th>Encashment</th>
<th>Accumulation</th>
<th>Approved By</th>
</tr>
<tr>
<td>Annual</td>
<td>29 days</td>
<td>On separation or retirement</td>
<td>10 per annum up to 65 days</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Sick</td>
<td>Open</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Parental Leave (Birth / Surrogacy / Adoption)</td>
<td>26 weeks</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Transition/Relo cation Leave</td>
<td>5 days</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Marriage Leave</td>
<td>2 days</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Bereavement Leave</td>
<td>2 days</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
<tr>
<td>Leave Without Pay</td>
<td>Maximum of 30 days</td>
<td>NA</td>
<td>NA</td>
<td>LF Head &amp; HR Business Partner</td>
</tr>
<tr>
<td>Compensatory Off</td>
<td>2 days</td>
<td>Nil</td>
<td>Nil</td>
<td>Immediate Manager</td>
</tr>
</table>


<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 1 of 7" -->
<!-- PageBreak -->


## Leave Guidelines

☐
For the purpose of calculating leave accounts, 'year' shall mean the year commencing
on the first day of April and ending on the last day of March next year ('Financial
year').

☐
] Discretion is reserved with the manager empowered to sanction leave, to refuse or
revoke leave at any time according to the exigencies of the organization's work.

☐
Leave must be applied for through HR Core.
☐
The associate is responsible for his/her attendance and leave records and should
always keep it updated.


## 1.1 Annual Leave (AL)

Entitlement
☐
All associates are entitled to 29 working days of AL for the financial year, credited on
1 st April every year. Associates joining during the year will be credited with leave on
a pro-rate basis for the remaining part of the financial year.

New Joiners joining on or before the 15th of the month are entitled to (2) two days
☐
AL for the first month and those joining after the 15th of the month are entitled to 1
(one) AL for the month.

Associates can avail a minimum of half day Annual Leave at a time.
☐

Accumulation

☐
In a given year, an associate can carry forward a maximum of 10 days of AL. The
balance AL of that year would automatically lapse on 31 st March.
☐
For hires during the leave calendar year, the leave accumulation will be pro-rated up
to a maximum of 10 days.

☐
AL can be accumulated up to a maximum of 65 days in total. Any leave balance in
excess of 65 days will lapse without any encashment.
☐
] For associates joining during the year, the accumulation of leaves will be pro-rated
up to a maximum limit of 10 days.

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 2 of 7" -->
<!-- PageBreak -->


# Encashment

☐
Encashment of leaves will be calculated on Base Salary' (not basic salary) as per the
formula "Base Salary / 26 * no. of leaves"

Guidelines
☐
Intervening Holidays do not form part of the Annual Leave.
☐
AL can be combined with Sick Leave.
☐
If associate could not make use of the leave after it was sanctioned, he/she is required
to cancel the leave giving reasons to their manager in the system.
☐
[ On resignation/superannuation of services from the company, excess AL in the
current year, against entitlement, will be adjusted in the full & final settlement on a
pro-rata basis subject to a cap of 10 days AL.


## 1.2 Sick Leave

Objective & Eligibility

To give associates time off to recover and recuperate in the event of illness or accidents.
All internal associates are eligible for this policy.


### Procedures and Conditions

☐
The company may, at its discretion, grant sick leave to an associate, based on the
nature of sickness.
☐

[ In case of absence beyond 3 days, the associate is required to submit a medical
certificate.

[ In case where absence is likely to be prolonged beyond 3 days, the likely duration of
☐

absence along with the doctor's certificate certifying the same should be submitted.

[ Any sick leave in excess of 30 continuous days will require the approval of the division
☐

management team. The management team will review the case and may at its
discretion approve additional sick leave with pay or without pay (Loss of Pay - LOP).

[ The company may request the associate to undergo a medical examination by a
☐

nominated medical practitioner if felt necessary.
☐
Sick leaves can neither be accumulated nor encashed nor carried forward to the next
year.

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 3 of 7" -->
<!-- PageBreak -->


### 1.3 Parental Leave

Both Birthing mother and Non-birthing parent are eligible for Parental Leave of 26 weeks
following the birth, surrogacy or adoption of a child.

Associates must avail the Parental Leave within one year of child's birth/surrogacy/adoption.
Any Parental Leave that is not used within one year of a covered life event will be forfeited
and will not be financially compensated, or carried forward. For details, refer to the policy
document - Parental Leave Policy - India


# Leave Policy for Trainees

Interns are eligible for a maximum of one day per month during their internship period in
case of medical and personal exigencies, subject to Manager's approval.


## Guidelines

☐
A minimum of one and maximum leave is at the discretion of the Manager.
☐
Intervening Holidays do not form part of the leave

☐
Any leave taken in excess of these 15 days in a given financial year would be treated
as leave without pay.


# Leave Without Pay (LWP)


## Eligibility

Leave without Pay (LWP) will be given to associates only in exceptional circumstances purely
at the discretion of the management. LWP can be availed only AFTER exhausting the Annual
Leave


## Process

Any associate planning to go on LWP will need to apply for the same to the immediate
Manager which would be approved by the SLT member and HR.


## Entitlement

Maximum 30 days Leave Without Pay in a year, at the discretion of the SLT member.

Unauthorised Absence

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 4 of 7" -->
<!-- PageBreak -->

☐
Unauthorized absence refers to absence from work without requisite approval.
☐
The associate will need to offer an explanation to $\mathrm { h i s / h e r }$ immediate manager in the
event of any unauthorized absence.
☐
In the event of an associate failing to give an explanation to the satisfaction of the

immediate manager, the employment is liable to termination as per organization
rules.

The associate will not be eligible for payment of salary during this absence.
☐


### General Compliance on Leave Rules

☐
Each associate is responsible for maintaining his/her updated leave record with all
leaves availed by him/her during the year on HR Core.

An associate on leave may be recalled to duty if an exceptional situation arises, and
the Company deems fit to do so.
☐
An associate going out of station during his/her leave must furnish to the manager
the contact numbers at which he/she can be contacted while out of station.
☐


### 1.4 Transition/Relocation Leave

Objective & Eligibility

Applicable only for those Associates who relocate (at the time of joining / internal
transfer) from other cities, it aims to provide them some time off to comfortably relocate
and settle in the new location.

Procedures and Conditions

☐
Eligible Associates will be entitled to Five (5) working days of Relocation Leave.
☐
This leave must be availed within Six (6) months from the date of joining / transfer
to the new location.
☐
The leave can be availed continuously or as and when required.

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 5 of 7" -->
<!-- PageBreak -->


### 1.5 Marriage Leave

Entitlement
Associates can avail 2 days leave at the time of own marriage


#### Conditions

2 days need to be taken consecutively at the time of marriage. If the marriage falls over the
weekend, marriage leave can be clubbed with the weekend.


# 1.6 Bereavement Leave

Entitlement

In the unfortunate event of the death of a parent, sibling, spouse or child, associates may
avail 2 days leave within a period of 3 months of the event.


## 1.7 Compensatory Offs

Objective & Eligibility

In order to meet business exigency associates have to come to work on weekly off or
public holidays. Associates who work on public holidays / weekly off day in case of
exigencies of work will be provided with a compensatory off. Compensatory off is allowed
subject to approval from Operational Manager

Eligibility: All associates at GJFA 5 to 8

Procedures and Conditions

1\. An associate should have worked at least for 8 hours on any off day to avail
compensatory off for the same.

2\. The compensatory off must be availed within two months.

3\. The compensatory off cannot be taken in advance.

4\. The compensatory off will not be carried forward or accumulated or en-cashed.

5\. Maximum 2 compensatory offs can be availed in a month

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 6 of 7" -->
<!-- PageBreak -->


### 1.8 Periodical Review of Leave Records

It is the responsibility of the manager to ensure that the leave records of their respective
teams are periodically reviewed and any discrepancies identified should be immediately
brought to the notice of the HRBP. HRBPs will then initiate corrections, if required, by
sending it to the HR Services team (XDHR) or through the employee/manager on HR
Core directly. In case any leaves are not recorded, managers should ensure that the
leaves are recorded on HR Core. Sufficient evidences of periodical review of leave records
should be maintained by the managers for their teams.

<!-- PageFooter="Novartis Hyderabad Business use only" -->
<!-- PageFooter="Back to Contents" -->
<!-- PageNumber="Page 7 of 7" -->


### Processing Insurance Policy Document

In [8]:
md_result, result_with_image_descp = run_document_intelligence(INPUT_FILE_PATH2)

# Save results to a single pickle file as a dictionary
results_dict = {
    "md_result": md_result,
    "result_with_image_descp": result_with_image_descp
}

with open(os.path.join(PICKLE_DIR, "insurance_policy_results.pkl"), "wb") as f:
    pickle.dump(results_dict, f)

Markdown(result_with_image_descp)

<!-- PageHeader="AON" -->


# Employee Benefits Manual 2023-24 Novartis Group


<figure>
</figure>


<!-- PageFooter="Aon India Insurance Brokers Private Limited (Formerly Anviti Insurance Brokers Private Limited) CIN U66000KA2016PTC095559, Composite Insurance Broker, IRDAI Licence No.624, Valid till 15/10/2023, aon.com Registered Office - Unit 102, 1st Floor, The Estate, 121 Dickenson Road, Bengaluru 560042" -->
<!-- PageBreak -->


## Group Hospitalization Policy

The Group Medical policy covers
expenses for the insured (employee &
eligible family members) on account of
hospitalization due to sickness or
accident. The policy covers expenses
incurred on room rent, medicines, surgery
etc. Expenses for hospitalization are
payable only if a 24-hour hospitalization
has been taken. Group Mediclaim is an
annual plan effective from 1st July 2023
to 30th June 2024; terms and conditions
are subject to renewal on annual basis.

8

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2022-23" -->

2

<!-- PageFooter="AON" -->
<!-- PageBreak -->


## Make The Best Use Of Your Benefits


<figure>


The process for getting enrolled involves planning the enrollment and following the process to obtain an E-Card. It is essential to ensure that all dependent details are accurately listed, including name, date of birth, gender, age, and relationship information. All this information must be correctly captured in the HR CORE system, which is the Novartis HR Database. This ensures that the enrollment process is complete and all necessary data is available for issuing the E-Card.
Getting Enrolled

· Plan Enrollment &
Process for getting
your E-Card

O

Please ensure that all your
dependent details are listed, and
their corresponding information
provided Name, DOB, Gender,
Age, and relationship details etc.
are captured correctly in HR
CORE (Novartis HR Database)

</figure>


Benefit Plan Details

· Plan Information,
Benefit Details &
General Exclusions

O

Please refer Employee
Insurance Benefit Manual for
information on Coverage,
exclusions & do save
emergency contact points, this
will be very handy in case of
emergency.


<figure>


The process of making a claim involves understanding the hospitalization procedure, reviewing the claims checklist, and referring to key frequently asked questions. It is essential to submit all required documents in their entirety at the first opportunity, ensuring that original documents are provided. Failure to submit complete documentation may result in the claim being closed. It is also important to keep copies of all documents submitted to the insurer for personal records.
Making A Claim

· Hospitalization
Procedure, Claims
Check List and Key
FAQS

O

Please ensure that your documents are
submitted entirely at the first instance
itself and originals are submitted.
Incomplete submissions may lead to
the claim getting closed. Please also
retain a copy of all claim documents
submitted to the insurer.

</figure>


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="3" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<td colspan="2">Plan Name : Group Mediclaim Policy</td>
</tr>
<tr>
<td>Policy Holder</td>
<td>Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Policy period</td>
<td>01-Jul-2023 To 30-Jun-2024</td>
</tr>
<tr>
<td>Insurer</td>
<td>Aditya Birla Health Insurance Co. Ltd</td>
</tr>
<tr>
<td>TPA</td>
<td>Medi Assist India TPA Pvt. Ltd.</td>
</tr>
<tr>
<td>Sum Insured Limits</td>
<td>Uniform INR 7 Lacs</td>
</tr>
<tr>
<td>Geographical Limits</td>
<td>India</td>
</tr>
<tr>
<td>Family Definition</td>
<td>· Single Associates - Self and 2 Dependent Parents or Partner · Married Associates - Self, Spouse or Partner and All Dependent Children upto 25 years ("Partner" includes married spouse, Domestic Partner (unmarried same sex or opposite sex) If Husband and Wife both are working with Novartis Group, family definition will differ specifically for those i.e. one employee can cover spouse and other can cover parents.</td>
</tr>
<tr>
<td>Mid-Term Enrollment</td>
<td>Allowed only for New Joiner, Newly Married Spouse &amp; Newborn Baby</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="4" -->
<!-- PageBreak -->


## Your Plan Details Family Definition


<table>
<tr>
<th>Particular</th>
<th>Included</th>
<th>Special Condition if any</th>
</tr>
<tr>
<td>Employee</td>
<td>Yes</td>
<td>-</td>
</tr>
<tr>
<td>Spouse</td>
<td>Yes</td>
<td>-</td>
</tr>
<tr>
<td>Child</td>
<td>Yes</td>
<td>· All dependent children till age of 25 years · No age limit for differently abled children · Legally adopted children of single employees/ same sex LGBT employees</td>
</tr>
<tr>
<td>Parents</td>
<td>Yes</td>
<td>Only for unmarried employees Parents over the age of 85 are covered only if they were covered in previous year policy</td>
</tr>
<tr>
<td>Live -in/same sex LGBT Partner</td>
<td>Yes</td>
<td>Only for unmarried employees after due declaration submission and if parents are not covered</td>
</tr>
<tr>
<td>Parent-in-Laws</td>
<td>No</td>
<td>-</td>
</tr>
<tr>
<td>Sibling</td>
<td>No</td>
<td>-</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="5" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<td>Standard Hospitalization</td>
<td>Covered</td>
</tr>
<tr>
<td>Pre-existing Diseases</td>
<td>Covered</td>
</tr>
<tr>
<td>Restriction on Room-Rent</td>
<td>For Normal-1% of Sum insured or single private AC room whichever is higher For ICU- As per actuals</td>
</tr>
<tr>
<td>Maternity Benefits</td>
<td>Covered</td>
</tr>
<tr>
<td>Maternity Limits</td>
<td>INR 75,000 for Normal &amp; INR 1 Lac (New) for Caesarean Section to only 2 deliveries</td>
</tr>
<tr>
<td>Pre &amp; Post Natal Expenses</td>
<td>Covered up to INR 5,000 , over and above the maternity limit, to only 2 deliveries</td>
</tr>
<tr>
<td>Newborn Baby Cover</td>
<td>Covered from day one subject to intimation to HR within 15 days from date of birth</td>
</tr>
<tr>
<td>Congenital Disease</td>
<td>Internal Congenital - Covered External Congenital - Covered in Life Threatening Situations only</td>
</tr>
<tr>
<td>Dental Benefit</td>
<td>Covered only in Accident cases (Hospitalization)</td>
</tr>
<tr>
<td>Critical Illness</td>
<td>Covered on voluntary basis</td>
</tr>
<tr>
<td>Diagnostics Expenses</td>
<td>Standalone diagnostic not covered</td>
</tr>
<tr>
<td>Ambulance Services</td>
<td>Local emergency Ambulance covered up to INR 3,000/- per person per policy. Ambulance cover to hometown for carriage of mortal remains of employee, covered up to INR 100,000</td>
</tr>
<tr>
<td>Non-Medical Expense</td>
<td>In the event of an employee's death during treatment or procedure, the plan will cover the entire cost of the including non-medical charges, otherwise, the associate is typically responsible for non- medical expenses</td>
</tr>
</table>


<figure>

AON

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="6" -->
<!-- PageBreak -->


## Your Plan Details Policy Benefits


<table>
<tr>
<td>Pre &amp; Post Hospitalization Expenses</td>
<td>30 days prehospitalization and 60 days post hospitalization.</td>
</tr>
<tr>
<td>Claims Submission Timeframes</td>
<td>Submission within 30 days from Date of Discharge</td>
</tr>
<tr>
<td>Claims Intimation Timeframes</td>
<td>Waived off</td>
</tr>
<tr>
<td>Therapy area brief - Plaque Psoriasis, Psoriatic Arthritis &amp; Ankylosing Spondylitis</td>
<td>covered up to INR 2,00,000 per family on OPD &amp; IPD basis</td>
</tr>
<tr>
<td>Refractive error or sight correction $\left( + / - \right) 7$</td>
<td>For Myopia, Astigmatism and Hypermetropia) covered with a cap of INR 50,000/- family limit</td>
</tr>
<tr>
<td>Macular Degeneration of Retina</td>
<td>Macular Degeneration of Retina (covering specific macular Indications - wAMD, DME, Diabetic Retinopathy , Retinal Vein Occlusion (cRVO and bRVO) and Retinopathy of prematurity) covered up to 50% of the sum insured either as in patient or as part of day care treatment</td>
</tr>
<tr>
<td>Gender Dysphoria (Gender change)</td>
<td>covered within family sum insured limits</td>
</tr>
<tr>
<td>Migraine management</td>
<td>Treatment using Monoclonal Antibodies for Episodic and Chronic Migraine patients with a capped at payout of INR 125,000 per family</td>
</tr>
</table>


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="7" -->
<!-- PageBreak -->


## Your Plan Details New Age Treatment covered under policy

$$N e w \quad A g e \quad T r e a t m e n t \quad o r \quad P r o c e d u r e$$


<table>
<tr>
<th></th>
<th>Limit as per (Per Policy Period)</th>
</tr>
<tr>
<td>Uterine Artery Embolization and HIFU (High intensity focused ultrasound)</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Balloon Sinuplasty</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Deep Brain stimulation</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Oral chemotherapy</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Immunotherapy- Monoclonal Antibody to be given as injection</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Robotic surgery</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Stereotactic radio surgery</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Bronchial Thermoplasty</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Vaporisation of the prostrate (Green laser treatment or holmium laser treatment)</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>IONM - (Intra Operative Neuro Monitoring)</td>
<td>Upto 50% of Sum Insured</td>
</tr>
<tr>
<td>Stem cell therapy: Hematopoietic stem cells for bone marrow transplant for haematological conditions to be covered</td>
<td>Upto 50% of Sum Insured</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="8" -->
<!-- PageBreak -->


## Your Plan Details What Is Covered?

If any Insured Person suffers an Illness or Accident during the Policy Period that requires Insured Person's hospitalization as an inpatient,
then the insurer will reimburse reasonable and customary expenses towards the below mentioned hospitalization under your group
medical plan.

· Inpatient Treatment

· Room rent and boarding expenses

· Doctor's fees ( who needs to be a medical practitioner)

· Intensive Care Unit

· Nursing expenses, Anesthesia, blood, oxygen, operation theatre charges, surgical appliances,

· Medicines, drugs and consumables (Dressing, ordinary splints and plaster casts)

· Diagnostic procedures (such as laboratory, x-ray, diagnostic tests)

· Costs of prosthetic devices if implanted internally during a surgical procedure

· Organ transplantation including the treatment costs of the donor but excluding the costs of the organ

The expenses shall be reimbursed provided they are incurred in India and are within the policy period. Expenses will be reimbursed
to the covered member depending on the level of cover that he/she is entitled to. Expenses that are of a diagnostic nature only or
are incurred from a preventive perspective with no active line of treatment and do not warrant a hospitalization admission are not
covered under the plan.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="9" -->
<!-- PageBreak -->


## Your Plan Details Pre & Post Hospitalization Expenses


<table>
<tr>
<td colspan="2">Pre-hospitalization expenses</td>
</tr>
<tr>
<td>Pre-hospitalization Expenses</td>
<td>If the Insured Person is diagnosed with an Illness which results in his or her Hospitalization and for which the Insurer accepts a claim, the Insurer will reimburse the Insured Person's Pre-hospitalization Expenses for up to 30 days prior to his Hospitalization as long as the 30-day period commences and ends within the Policy Period.</td>
</tr>
<tr>
<td>Duration</td>
<td>Within 30 days before hospitalization</td>
</tr>
</table>


<table>
<tr>
<td colspan="2">Post hospitalization expenses</td>
</tr>
<tr>
<td>Post-hospitalization Expenses</td>
<td>If the Insurer accepts a claim above and, immediately following the Insured Person's discharge, he requires further medical treatment directly related to the same condition for which the Insured Person was Hospitalized, the Insurer will reimburse the Insured Person's Post-hospitalization Expenses</td>
</tr>
<tr>
<td>Duration</td>
<td>Within 60 days post discharge</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="10" -->
<!-- PageBreak -->


## Your Plan Details Maternity Benefits

· Maternity benefits are admissible only if the expenses are incurred in Hospital / Nursing Home as in-patients in India.

· Those Insured Persons who already have two or more living children will not be eligible for this benefit.

· Expenses incurred in connection with voluntary medical termination of pregnancy during the first 12 weeks from the date
of conception are not covered. Infertility Treatment and sterilization are excluded from the policy.


### The maternity benefit under your Group Medical Plan


<table>
<tr>
<td>Maximum Benefit</td>
<td>INR 75,000 for Normal &amp; INR 1 Lac (New) for C - Section within Sum Insured Limit</td>
</tr>
<tr>
<td>9-months waiting period</td>
<td>Waived off</td>
</tr>
<tr>
<td>Pre-Post Natal expenses</td>
<td>Covered up to INR 5,000 over and above the maternity limit</td>
</tr>
<tr>
<td>Newborn baby covered from day 1</td>
<td>Covered from day 1, subject to intimation to HR within 15 days from date of birth</td>
</tr>
</table>


IMPORTANT :

For maternity reimbursements and employees on subsequent maternity leave, please do not wait till you have returned
back to office to submit a claim as it will cross the claim submission within 30 days to avoid denial of claim. Please also
immediately inform your HR about the new baby coverage as your dependent as A subsequent complication may be A
possibility and intimation is mandatory prior to coverage


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="11" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<th>Policy Benefit</th>
<th>Definition</th>
<th>Covered/Not Covered</th>
</tr>
<tr>
<td>Pre-existing Diseases</td>
<td>Any Pre-Existing Condition or related condition for which care, treatment or advice was recommended by or received from a Doctor or which was first manifested prior to the commencement date of the Insured Person's first Health Insurance policy with the Insurer</td>
<td>Covered</td>
</tr>
<tr>
<td>First 30 days' waiting period</td>
<td>Any Illness diagnosed or diagnosable within 30 days of the effective date of the Policy Period if this is the first Health Policy taken by the Policyholder with the Insurer.</td>
<td>Covered</td>
</tr>
<tr>
<td>First Year Waiting Period</td>
<td>During the first year of the operation of the policy the expenses on treatment of diseases such as Cataract, Benign Prostatic Hypertrophy, Hysterectomy for Menorrhagia or Fibromyoma, Hernia, Hydroceie, Congenital Internal Diseases, Fistula in anus, Piles, Sinusitis and related disorders are not payable. If these diseases are pre- existing at the time of proposal, they will not be covered even during subsequent period or renewal too</td>
<td>Waived off</td>
</tr>
<tr>
<td>Day Care</td>
<td>Day Care Procedure means the course of medical treatment, or a surgical procedure listed in the Schedule which is undertaken under general or local anesthesia in a Hospital by a Doctor in not less than 2 hours and not more than 24 hours.</td>
<td>Covered</td>
</tr>
<tr>
<td>Diagnostic Expenses</td>
<td>All diagnostic tests and lab tests as part of hospitalization and pre-post hospitalization including OPD. Diagnostic tests without treatment or not related to treatment are not covered</td>
<td>Covered and Only incase of 24-hr hospitalization related to treatment</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="12" -->
<!-- PageBreak -->


## Your Plan Details


### How to Enroll?


<table>
<tr>
<th>Particular</th>
<th>Description</th>
<th>Special Condition if any</th>
</tr>
<tr>
<td>Enrollment of existing employees' and their dependents</td>
<td>Enrollment/Validation</td>
<td>Employee to either declare new dependents or validate current dependent details at the time of renewal every year</td>
</tr>
<tr>
<td>Enrollment of New Joinees (New Employees + their Dependents) during the policy year</td>
<td>Declaration</td>
<td>Dependent information needs to be updated with 30 days from date of joining on HR system</td>
</tr>
</table>


### Mid Term Enrollment Eligibility?


<table>
<tr>
<th>Particular</th>
<th>Description</th>
<th>Special Condition if any</th>
</tr>
<tr>
<td>Mid-Term Enrollment of Existing employees' Dependents(as on plan start date)</td>
<td>Not Allowed</td>
<td></td>
</tr>
<tr>
<td>Mid-Term Enrollment of New Joinees (New Employees +Their Dependents)</td>
<td>Allowed *</td>
<td></td>
</tr>
<tr>
<td>Mid-Term Enrollment of New Dependents (Spouse/Children)</td>
<td>Allowed *</td>
<td>Midterm enrollment of new dependents (Spouse / Children) is allowed for employees within 30 days from Date of Marriage/ Date of Birth.</td>
</tr>
<tr>
<td>Mid term inclusion of Parents</td>
<td>Allowed</td>
<td>Only in case of legal divorce, death of spouse and if no children are enrolled in the policy, required documents have to be submitted to respective HR spocs</td>
</tr>
</table>


<figure>

AON

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="13" -->
<!-- PageBreak -->


## Getting Enrolled Process for Domestic Partner Enrolment

· DOMESTIC PARTNERSHIP* means a committed relationship between two adults (same or opposite sex),
that meets all of the requirements as per the attached declaration form

· Dependent Children: To qualify as a dependent child, the child must be legally adopted child of an
employee or his/her domestic partner should be a legal parent of child(ren), legal document to be
submitted as a part of the declaration process

· Please email the following below mentioned details to your respective HR SPOCs with duly filled and
signed Declaration Form

A. Your Employee Code

B. Full Name of the Partner

C. DOB of the Partner

D. Gender of the Partner


<figure>

PDF

Domestic
Declaration Form

</figure>


Please Note:

If you opt for the above partner enrolment, then in line with
the eligibility guidelines your parents shall no longer be
eligible for cover under the medical insurance plan and will
be removed from coverage .

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="14" -->
<!-- PageBreak -->


## Health E- Cards Process of Download E-Health Id Cards


<figure>

Medi Assist

</figure>


Test000

«Home

EMenu

[+ Sign out

· Enter URL in address bar of
your browser :
https://portal.medibuddy.in/lo
gin.aspx

. Sign in with Username and
Password as received via mail
from

. Click on the menu, located at
top right corner

. Click on "E Card" to download
pdf of your E 
A comprehensive health insurance portal interface is organized into distinct functional sections, each representing a key service or feature. The top row provides access to claims management, wellness resources, pharmacy services, health-related updates, and a hospital search tool. The central section focuses on understanding health coverage, with options to view the employee policy, initiate enrolment, and download electronic cards. The bottom row includes personal profile management, cashless service initiation, reimbursement processes, claim submission, and direct contact information for support, including a helpline number and an email option. Each section is clearly labeled, ensuring users can efficiently navigate to the desired service, whether for information, action, or assistance. The layout emphasizes user-centric access to all major health insurance functionalities, supporting both self-service and direct support channels.
cards.


<figure>

Wellness

Your claims

Pharmacy

Health bites

Search hospitals

Know
your
health
cover

Enrolment

Employee policy

Download E-cards

Contact us

eCashless

1800-425-9449

Write to us

Your profile

Intimate
Reimbursement

Submit claim

</figure>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="15" -->
<!-- PageBreak -->


## Voluntary Critical Illness - Plan Details Policy Benefits


<table>
<tr>
<td>Cover Definition</td>
<td>A policy that pays a lump sum amount equal to the sum insured upon first diagnosis of $a$ critical illness covered under the policy. In this case, 36 Critical Illnesses are covered</td>
</tr>
<tr>
<td>Sum Insured</td>
<td>INR 10 Lakhs</td>
</tr>
<tr>
<td>Cover Type</td>
<td>Individual cover of INR 10 Lakhs</td>
</tr>
<tr>
<td>Family Definition</td>
<td>Employee, Spouse &amp; Dependent Children (min 5 years age)</td>
</tr>
<tr>
<td>Waiting Period</td>
<td>90 Days</td>
</tr>
<tr>
<td>Survival Period</td>
<td>30 days</td>
</tr>
<tr>
<td>Pre-Existing Diseases</td>
<td>Not covered</td>
</tr>
<tr>
<td>Number of Critical Illness cover</td>
<td>36 Critical Illness covered (separate slide of covers in detail)</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="16" -->
<!-- PageBreak -->


## Critical Illness (1/2) List of New 36 Critical Illness


<table>
<tr>
<th>Cancer of specific severity</th>
<th>Multiple Sclerosis with Persisting Symptoms</th>
</tr>
<tr>
<td>Myocardial Infarction</td>
<td>Coma of Specified Severity</td>
</tr>
<tr>
<td>Open Chest CABG</td>
<td>Motor Neurone Disease with Permanent Symptoms</td>
</tr>
<tr>
<td>Open Heart Replacement or Repair of Heart Valves</td>
<td>Loss of Vision (Blindness)</td>
</tr>
<tr>
<td>Kidney Failure Requiring Regular Dialysis</td>
<td>Major burns</td>
</tr>
<tr>
<td>Stroke Resulting in Permanent Symptoms</td>
<td>Parkinson's Disease</td>
</tr>
<tr>
<td>Major Organ / Bone Marrow Transplant</td>
<td>Benign Brain Tumor</td>
</tr>
<tr>
<td>Permanent Paralysis of Limbs</td>
<td>Alzheimer's Disease</td>
</tr>
<tr>
<td>Bacterial Meningitis</td>
<td>Apallic Syndrome or Persistent Vegetative State (PVS)</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="17" -->
<!-- PageBreak -->


## Critical Illness (2/2) List of New 36 Critical Illness


<table>
<tr>
<th>Aorta Graft Surgery</th>
<th>Encephalitis</th>
</tr>
<tr>
<td>Loss of Hearing (Deafness)</td>
<td>Fulminant Hepatitis</td>
</tr>
<tr>
<td>Loss of Limbs</td>
<td>Chronic Relapsing Pancreatitis</td>
</tr>
<tr>
<td>Loss of Speech</td>
<td>Major Head Trauma</td>
</tr>
<tr>
<td>Aplastic Anaemia</td>
<td>Medullary Cystic Disease</td>
</tr>
<tr>
<td>End Stage Liver Failure</td>
<td>Muscular Dystrophy</td>
</tr>
<tr>
<td>End Stage Lung Failure</td>
<td>Poliomyelitis</td>
</tr>
<tr>
<td>Primary (Idiopathic) Pulmonary Hypertension</td>
<td>Systemic Lupus Erythematous</td>
</tr>
<tr>
<td>Coronary Angioplasty (PTCA)[1]</td>
<td>Brain Surgery</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="18" -->
<!-- PageBreak -->


## Critical Illness General Exclusions


<table>
<tr>
<td>1</td>
<td>We shall not be liable to make any payment in respect of any Critical Illness whose signs or symptoms first occur within the first 90 days from the Inception Date</td>
</tr>
<tr>
<td>2</td>
<td>Any claim with respect to any Critical Illness diagnosed or which manifested prior to the Inception Date</td>
</tr>
<tr>
<td>3</td>
<td>Any Pre-Existing Disease or any complication arising therefrom</td>
</tr>
<tr>
<td>4</td>
<td>Any Critical Illness arising out of use, abuse or consequence or influence of any substance, intoxicant, drug, alcohol or hallucinogen</td>
</tr>
<tr>
<td>5</td>
<td>Any Critical Illness directly or indirectly caused due to intentional self-injury, suicide or attempted suicide; whether the person is medically sane or insane</td>
</tr>
<tr>
<td>6</td>
<td>Working in underground mines, tunneling or involving electrical installations with high tension supply, or as jockeys or circus personnel</td>
</tr>
<tr>
<td>7</td>
<td>Any loss resulting directly or indirectly contributed or aggravated or prolonged by childbirth or from pregnancy</td>
</tr>
<tr>
<td>8</td>
<td>Any Critical Illness arising or resulting from the Insured Person committing any breach of law or participating in an actual or attempted felony, riot, crime, misdemeanor or civil commotion with criminal intent</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="19" -->
<!-- PageBreak -->


## Voluntary Parent's Policy- Plan Details


<table>
<tr>
<td colspan="2">Plan Name : Voluntary Parent's Mediclaim Policy</td>
</tr>
<tr>
<td>Policy Holder</td>
<td>Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Policy period</td>
<td>01-Jul-2023 To 30-Jun-2024</td>
</tr>
<tr>
<td>Insurer</td>
<td>Aditya Birla Health Insurance Co. Ltd</td>
</tr>
<tr>
<td>TPA</td>
<td>Medi Assist India TPA Pvt. Ltd.</td>
</tr>
<tr>
<td>Sum Insured Limits</td>
<td>INR 3 Lacs &amp; INR 5 Lacs</td>
</tr>
<tr>
<td>Geographical Limits</td>
<td>India</td>
</tr>
<tr>
<td>Family Definition</td>
<td>. Parents or Parent In Laws . Cross combination not allowed Parents here refers the Primary Insured's natural parents or parents that have legally adopted him/her</td>
</tr>
<tr>
<td>Entry Age for Parents</td>
<td>For Parents/ in laws- Entry age restricted upto 85 years</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="20" -->
<!-- PageBreak -->


## Voluntary Parent's Policy - Plan Details (12)


<table>
<tr>
<td>Room Rent Restriction</td>
<td>2% of the in-patient Sum Insured, if admitted to regular room. All benefits as an inpatient in a hospital attached to room will be restricted to the room, which falls within the room rent limits allowed. The enhanced difference in expenses due to opting rooms with higher room rent than what has been allowed will be borne by the insured only. Wherever the room rent based tariff for the other expenses is not available, the payment would be done in the same proportion as per the entitlement of room rent under the policy excluding medicines, consumables and implants medically prescribed by the treating doctor under the policy. In case of package treatment where individual bifurcation of room rent, medicines, operation theatre expenses, doctor's consultation charges etc. are not available, then the package charges shall be proportionately linked to the entitled room rent of the insured person under the policy.</td>
</tr>
<tr>
<td>Pre &amp; Post Hospitalization expenses</td>
<td>30 Days Pre-Hospitalization &amp; 60 days Post-Hospitalization (to be claimed as reimbursements)</td>
</tr>
<tr>
<td>Road Ambulance Charges</td>
<td>Covered upto INR 2500 per case in case of emergency only. Ambulance charges will be applicable for transferring patient to Hospital or between Hospitals in the Hospitals ambulance or in an ambulance provided by any ambulance service provider only</td>
</tr>
<tr>
<td>Internal Congenital</td>
<td>Covered</td>
</tr>
<tr>
<td>Cataract Surgery</td>
<td>Mono and/or Multifocal Lens covered</td>
</tr>
<tr>
<td>Survival Period</td>
<td>30 days</td>
</tr>
<tr>
<td>Co-Pay</td>
<td>10% Co-Pay on each and every claim</td>
</tr>
</table>


<figure>

AON

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="21" -->
<!-- PageBreak -->


## Voluntary Parent's Policy - Plan Details (2/2)


<table>
<tr>
<td>Special Conditions</td>
<td>Pandemic and Dental and Vision in cases of accidents covered, only in case of accidents. Refractive error or sight correction $\left( + / - \right) 7$ and macular degeneration of retina- on IPD and OPD basis, Final decision will be taken on receipt of complete set of documents. 50% Co-Pay for cyber-knife treatment, Gamma Knife treatment and Stem Cell Transplantation, Robotic Surgery, Femto laser treatment for eye. It will be applicable for each eye each event. Cochlear Implant treatment shall be restricted to 50% of the SI. Coverage under this benefit is subject to the procedure being authority-approved indication for the medical condition, and should not be experimental and unproven treatment as on date</td>
</tr>
<tr>
<td>Pre-Existing exclusions/1st year exclusions/30 days waiting period</td>
<td>Waived off</td>
</tr>
<tr>
<td>Claims Submission Timeframes</td>
<td>Submission within 30 days from Date of Discharge</td>
</tr>
<tr>
<td>Claims Intimation Timeframes</td>
<td>Waived off</td>
</tr>
<tr>
<td>Terrorism</td>
<td>Covered</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="22" -->
<!-- PageBreak -->


## Know Your Room Rent Limit Proportionate Deduction Clause

· Insured employees are requested to use prudence and proper negotiation with Hospital/ Nursing home in availing
the eligible room category.

. Please remember, higher the room category higher is the cost of treatment. This may result in faster exhaustion
of your total available eligibility

· Employee opting for a higher room category will have to bear the proportionate increase in cost on all categories /
heads.

. For example: if an employee opts for room of INR 10,000 per day , 50% deduction shall be applicable for overall
bill except for the cost of medicines.

· Any expenses like nursing / electricity / monitor charges / etc. charged separately will be considered under room
rent cost and will not be payable if charged separately.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="23" -->
<!-- PageBreak -->


## Your Plan Details(1/3) General Exclusions

· Injury or disease directly or indirectly caused by or arising from or attributable to War, Invasion, Act of Foreign Enemy, War like
operations (whether war be declared or not) or by nuclear weapons / materials.

· Circumcision (unless necessary for treatment of a disease not excluded hereunder or as may be necessitated due to any accident),
vaccination, inoculation or change of life or cosmetic or of aesthetic treatment of any description, plastic surgery other than as may
be necessitated due to an accident or as a part of any illness.

· Surgery for correction of eye- sight, cost of spectacles, contact lenses, hearing aids etc.

· Cosmetic or of aesthetic procedure, filling of cavity, root canal including wear and tear etc unless arising from disease or injury and
which requires hospitalization for treatment.

· Congenital external diseases or defects/anomalies

· Convalescence, general debility, "run down" condition or rest cure, congenital external diseases or defects or anomalies, sterility,
any fertility, sub-fertility or assisted conception procedure, venereal diseases, intentional self-injury/suicide, all psychiatric and
psychosomatic disorders and diseases / accident due to and or use, misuse or abuse of drugs / alcohol or use of intoxicating
substances or such abuse or addiction etc.

. Any cosmetic or plastic surgery except for correction of injury

· Expenses incurred at Hospital or Nursing Home primarily for evaluation / diagnostic purposes which is not followed by active
treatment for the ailment during the hospitalized period.

· Expenses on vitamins and tonics etc unless forming part of treatment for injury or disease as certified by the attending physician.

· Any Treatment arising from or traceable to pregnancy, miscarriage, abortion or complications of any of these including changes in
chronic condition as a result of pregnancy except where covered under the maternity section of benefits.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="24" -->
<!-- PageBreak -->


## Your Plan Details(2/3) General Exclusions

· Doctor's home visit charges, Attendant / Nursing charges during pre and post Hospitalization period.

· Treatment which is continued before hospitalization and continued even after discharge for an ailment / disease / injury different
from the one for which hospitalization was necessary.

· Naturopathy treatment, unproven procedure or treatment, experimental or alternative medicine and related treatment including
acupressure, acupuncture, magnetic and such other therapies etc.

· External and or durable Medical / Non-medical equipment of any kind used for diagnosis and or treatment including CPAP, CAPD,
Infusion pump etc., Ambulatory devices i.e. walker , Crutches, Belts , Collars , Caps , splints, slings, braces , Stockings etc of any
kind, Diabetic footwear, Glucometer / Thermometer and similar related items etc and also any medical equipment which is
subsequently used at home etc ..

. All non-medical expenses including Personal comfort and convenience items or services such as telephone, television, Aya /
barber or beauty services, diet charges, baby food, cosmetics, napkins, toiletry items etc, guest services and similar incidental
expenses or services etc ..

· Change of treatment from one pathy to other pathy unless being agreed / allowed and recommended by the consultant under whom
the treatment is taken.

· Treatment of obesity or condition arising therefrom (including morbid obesity) and any other weight control programme, services or
supplies etc ..

· Any treatment required arising from Insured's participation in any hazardous activity including but not limited to scuba diving, motor
racing, parachuting, hang gliding, rock or mountain climbing etc unless specifically agreed by the Insurance Company.

. Any treatment received in convalescent home, convalescent hospital, health hydro, nature care clinic or similar establishments.

· Any stay in the hospital for any domestic reason or where no active regular treatment is given by the specialist.

· Massages, Steam bathing, Shirodhara not covered.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="25" -->
<!-- PageBreak -->


## Your Plan Details(3/3) General Exclusions

. Any kind of Service charges, Surcharges, Admission fees / Registration charges etc levied by the hospital.

. Non-prescribed drugs and medical supplies, Hormone replacement therapy

· Expenses incurred for investigation or treatment irrelevant to the diseases diagnosed during Hospitalization or primary reasons for
admission. Private nursing charges, Referral fee to family doctors, Out station consultants / Surgeon fees etc,.

· Vitamins and tonics unless used for treatment of injury or disease

· Infertility treatment, Intentional self Injury, Outpatient treatment.

· Family planning Operations (Vasectomy or tubectomy) etc

. All expenses arising out of any condition directly or indirectly caused by or associated with Human T-cell Lymphotropic Virus Type III
(HTLD - III) or Lymohadinopathy Associated Virus (LAV) or the Mutants Derivative or Variations Deficiency Syndrome or any
Syndrome or condition of similar kind commonly referred to as AIDS, HIV and its complications including sexually transmitted
diseases.

. External and or durable Medical / Non-medical equipment of any kind used for diagnosis and or treatment like Prosthetics etc.

. Any device/instrument/machine that does not become part of the human anatomy/body but would contribute/replace the function
of an organ is not covered.

· Warranted that treatments on trial/experimental basis are not covered under scope of the policy.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="26" -->
<!-- PageBreak -->


## Getting Enrolled The Procedure : What Must You Remember ?

. Employees have to provide all the details of dependents in the prescribed format provided in the joining docket for Mediclaim
coverage. Dependents once declared cannot be changed during the policy period.

· Existing Employees are covered as on date of policy commencement (or date of joining for new employees joining after 1st July 2022)
along with their eligible dependents as per data provided by HR to Insurance Company.

. No midterm inclusion of dependents would be allowed except in case of spouse due to marriage of an employee and birth of child.

· Midterm enrollment of new dependents (Spouse / Children) is allowed for employees within 15 days from Date of Marriage/ Date of
Birth. The details need to be provided to HR within 15 days from date of event.

. Eligible Dependent covered under the policy for existing employees can be viewed on the TPA website.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="27" -->
<!-- PageBreak -->


## Making A Claim Hospitalization Procedure

You can avail either cashless facility or submit the claim for reimbursement.


### Definition of Cashless

· Cashless hospitalization means the TPA may authorize (upon an Insured person's request) for direct settlement of eligible services
and the corresponding charges between a Standard Network / PPN Network Hospital and the TPA. In such case, the TPA will
directly settle all eligible amounts with the Network Hospital and the Insured Person may not have to pay any deposits at the
commencement of the treatment or bills after the end of treatment to the extent these services are covered under the Policy. Denial
of cashless does not mean that the treatment is not covered by the policy.


### Definition of Reimbursement

. In case you choose a non-network hospital, you will have to liaise directly with the hospital for admission. However, you are advised
to follow the preauthorization procedure and intimate the TPA about the claim to ensure eligibility for reimbursement of
hospitalization expenses from the insurer.

· To know about cashless or reimbursement, please visit the desired section mentioned below:

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="28" -->
<!-- PageBreak -->


## Making A Claim Process for Cashless

Cashless hospitalization means the Administrator may authorizes upon a Policyholder's request for direct settlement of
eligible services and its according charges between a Network Hospital and the Administrator. In such case the
Administrator will directly settle all eligible amounts with the Network Hospital and the Insured Person may not have to
pay any deposits at the commencement of the treatment or bills after the end of treatment to the extent as these
services are covered under the Policy.


<table>
<tr>
<td colspan="2">List of hospitals in the TPA's network eligible for cashless hospitalization</td>
</tr>
<tr>
<td>Hospital Network List</td>
<td></td>
</tr>
<tr>
<td>For Network Hospital List Click</td>
<td>Essential email ids and contact numbers:</td>
</tr>
<tr>
<td>https://www.medibuddy.in/networkHospitals</td>
<td></td>
</tr>
<tr>
<td>Note : Employee can always download the updated network list from TPA website on real time basis.</td>
<td rowspan="2">24x7 Toll Free Number for Claim Intimation: 040 6817 2735 Claim Intimation E-mail: Novartis@mediassist.in</td>
</tr>
<tr>
<td>For TPA Website Click:</td>
</tr>
<tr>
<td>https://portal.medibuddy.in/login.aspx</td>
<td></td>
</tr>
</table>


<figure>
</figure>



Information is provided for employees regarding the process to access the list of hospitals in the Third Party Administrator (TPA) network that are eligible for cashless hospitalization. Employees are instructed to view the network hospital list by visiting a specified online link. It is noted that the hospital network list can be downloaded and is updated in real time on the TPA website. Additionally, a separate link is provided for accessing the TPA website directly.

For support and claim-related queries, essential contact information is given. This includes a 24x7 toll-free number for claim intimation and an email address specifically for claim intimation purposes. The structure ensures that employees have immediate access to both the hospital network information and the necessary contact details for claim processing.
Please reach out to MediAssist TPA to check the admissibility of any claim in advance to avoid any ambiguity

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="29" -->
<!-- PageBreak -->


## Medi Assist TPA point of contacts

For any support in Cashless claims, Reimbursement claims, E-cards, network hospital support, cover details,
please contact Medi Assist TPA


<table>
<tr>
<th>Hyderabad</th>
<th>Mumbai</th>
</tr>
<tr>
<td>Helpline no - 040-68172735</td>
<td>Helpline no - 040-68213616</td>
</tr>
<tr>
<td>E-Mail Id - novartis@mediassist.in</td>
<td>E-Mail Id - novartis@mediassist.in</td>
</tr>
<tr>
<td>1st Level - K Raj Reddy - kancharla.reddy@mediassist.in</td>
<td>1st Level - Ramesh Baswal - ramesh.baswal@mediassist.in</td>
</tr>
<tr>
<td>9731847099</td>
<td>9515121283</td>
</tr>
<tr>
<td>2nd Level - Achanta Hari Tulasi - achanta.tulasi@mediassist.in</td>
<td>2nd Level - Achanta Hari Tulasi - achanta.tulasi@mediassist.in</td>
</tr>
<tr>
<td>9620856349</td>
<td>9620856349</td>
</tr>
<tr>
<td>3rd Level - Devendar Rao CH - devendar.rao@mediassist.in 6366420429</td>
<td>3rd Level - Devendar Rao CH - devendar.rao@mediassist.in 6366420429</td>
</tr>
</table>


Please reach out to MediAssist TPA to check the admissibility of any claim in advance to avoid any ambiguity

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="30" -->
<!-- PageBreak -->


## Making A Claim Group Medical- Cashless Hospitalization


<figure>


Planned hospitalization requires the patient to approach the hospital at least 48 hours before admission, present a TPA card along with a government ID, and complete pre-authorization formalities. The hospital then sends a pre-authorization letter to the TPA for approval. If all documents are in order, the TPA issues an authorization letter to the hospital within three hours. If the case is declined, a denial letter is issued to the hospital, but denial of cashless does not mean denial of treatment or claim. If additional information is needed, the TPA will inform the hospital or employee. This process ensures that all necessary documentation and approvals are managed efficiently before planned hospital admission.
Planned
Hospitalization

Approach hospital minimum 48 hrs. prior
to admission , produces TPA card with
Govt ID and completes pre-authorization
formalities

Faxes Pre-Authorization letter to TPA for
Approval

If all the documents are in order, TPA will
issue authorization letter to hospital
within 3 hours

If the case is Declined, Denial Letter will
be issued to hospital

(denial of cashless does not mean denial
of treatment or claim)

Incase additional information is required,
TPA will inform the Hospital / Employee

</figure>


<figure>


Emergency hospitalization involves a sequence of steps to ensure proper authorization and communication between the hospital, the Third Party Administrator (TPA), and the employee. The process begins with the patient’s admission to the hospital. Within 24 hours, pre-authorization formalities must be completed and submitted to the TPA for approval. If all required documents are in order, the TPA issues an authorization letter to the hospital within three hours. If the case is declined, a denial letter is issued, clarifying that denial of cashless service does not equate to denial of treatment or claim. If additional information is needed, the TPA will notify the hospital or employee accordingly. This structured process ensures timely decision-making and clear communication throughout the emergency hospitalization procedure.
Emergency
Hospitalization

Admission in Hospital

Pre-Authorization formalities to be
completed within 24 hours and sent to
TPA for Approval

If all the documents are in order, TPA
will issue authorization letter to hospital
within 3 hours

If the case is Declined, Denial Letter will
be issued

(denial of cashless does not mean
denial of treatment or claim)

Incase additional information is
required, TPA will inform the Hospital /
Employee

</figure>


For Cashless Network Hospital List Click

<!-- PageFooter="AON" -->
<!-- PageFooter="https://www.medibuddy.in/networkHospitals" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="31" -->
<!-- PageBreak -->


## Making A Claim Group Medical- Reimbursement 
The process for claiming insurance after treatment at a non-network hospital involves several sequential steps and decision points. Initially, the insured individual seeks treatment at a non-network hospital. After treatment, the individual is discharged and pays for the treatment out of pocket. The next step is to collect all original documents, receipts, and investigation reports from the hospital. These documents, along with a filled claim form, must be submitted to the Third Party Administrator (TPA) within 21 days from the date of discharge.

Once the TPA receives the claim documents, they acknowledge receipt via email and begin processing the claim. If additional information is required, the TPA will notify the employee via email and send reminders. If the required documents are not submitted within 30 days, the claim may be closed.

If the claim is found to be payable, the payment is made to the employee through NEFT (National Electronic Funds Transfer). If the claim is declined, a denial email is sent to the employee.

This process map clearly outlines the required actions, timelines, and decision points for both the insured and the TPA, ensuring transparency and clarity in the claim settlement process.
Claim


<figure>

Insured visits non network
hospital for treatment

Takes discharge, pays for
treatment

Collects all original
documents, receipts and
investigation reports from
Hospital

Submits all original
Hospital documents along
with filled claim form
within 21 days from date
of discharge to TPA

TPA acknowledges
receipt of claim
documents via email and
commences claim process

information is required, TPA
will inform the employee via
email with reminders, If
documents are not
submitted within 30 days,
claim may be closed/

If claim is payable,
payment will be made to
employee via NEFT

If claim is declined, denial
mail will be sent.

</figure>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="32" -->
<!-- PageBreak -->


## Making A Claim Claims Document Check List & Attachments


<table>
<tr>
<th>Sr. No.</th>
<th>Document Required (All in ORIGINAL)</th>
</tr>
<tr>
<td>1</td>
<td>Claim intimation is mandatory within 7 days from DOA and before DOD</td>
</tr>
<tr>
<td>2</td>
<td>Main Hospital bills in original (Original Hospital Payment Receipt with serial number, with bill no; signed and stamped by the hospital) &amp; itemized bills.</td>
</tr>
<tr>
<td>3</td>
<td>Discharge Card/Summary (original).</td>
</tr>
<tr>
<td>4</td>
<td>Attending doctors' bills and receipts and certificate regarding diagnosis (if separate from hospital bill) Indoor case papers</td>
</tr>
<tr>
<td>5</td>
<td>Original investigation reports or attested copies of Bills &amp; Receipts for Medicines, Investigations along with Doctors prescription in Original &amp; Laboratory</td>
</tr>
<tr>
<td>6</td>
<td>Follow-up advice or letter for line of treatment after discharge from hospital, from Doctor.</td>
</tr>
<tr>
<td>7</td>
<td>Break up with details of Pharmacy items, Materials, Investigations even though it is there in the main bill</td>
</tr>
<tr>
<td>8</td>
<td>In non- network hospitalization, please get the hospital and doctor's registration number in Hospital letterhead and get the same signed and stamped by the hospital.</td>
</tr>
<tr>
<td>9</td>
<td>In case of accidents, please note FIR or MLC (medico legal certificate) is mandatory. Original Death Summary: In case of Death Claims</td>
</tr>
<tr>
<td>10</td>
<td>One Cancelled Cheque with Employee Name printed to settle the claim to Employee Bank account.</td>
</tr>
</table>


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="33" -->
<!-- PageBreak -->


## Making A Claim Group Medical- Important Video Links- Medi Assist


<table>
<tr>
<th>Topic</th>
<th>Link</th>
</tr>
<tr>
<td rowspan="2">Hassle-free healthcare</td>
<td>https://www.youtube.com/watch?v=Fth</td>
</tr>
<tr>
<td>U5JU1QTU</td>
</tr>
<tr>
<td rowspan="2">One app for all your healthcare needs</td>
<td>https://www.youtube.com/watch?v=lhi</td>
</tr>
<tr>
<td>WAeAdY54</td>
</tr>
<tr>
<td>E-cashless to beat the que</td>
<td>https://youtu.be/urNNfFva864</td>
</tr>
<tr>
<td>Self-help anytime anywhere</td>
<td>https://youtu.be/QUowKC8jb3M</td>
</tr>
<tr>
<td>Paperless mobile on the go</td>
<td>https://youtu.be/l78q44WlDJY</td>
</tr>
<tr>
<td>Health benefits info at your fingertips</td>
<td>https://youtu.be/Cb0gpM3alDU</td>
</tr>
<tr>
<td rowspan="2">E-Cashless</td>
<td>https://youtu.be/3dgifO8QpWE</td>
</tr>
<tr>
<td></td>
</tr>
<tr>
<td>Network hospital search</td>
<td>https://youtu.be/U XPR7LYNvo</td>
</tr>
</table>


<table>
<tr>
<th>Topic</th>
<th>Link</th>
</tr>
<tr>
<td>MediBuddy Chatbot</td>
<td>https://youtu.be/0wfV5KfiAoQ</td>
</tr>
<tr>
<td>Real time Claims Tracking</td>
<td>https://youtu.be/U4ocgz0QnLk</td>
</tr>
<tr>
<td>Network hospital search with #MediBuddy #SelfHelp #Healthcare #Wellness #HealthTech</td>
<td>https://youtu.be/GbbZA964_zg</td>
</tr>
<tr>
<td>Online reimbursement claims submission</td>
<td>https://youtu.be/nycXXW2m56I</td>
</tr>
<tr>
<td>Pharmacy</td>
<td>https://youtu.be/kReYoxxJJH0</td>
</tr>
<tr>
<td>Ecard</td>
<td>https://youtu.be/OX2N7HCkr00</td>
</tr>
<tr>
<td>Maternity with #MediBuddy #MaternityProgram</td>
<td>https://youtu.be/651LEr1ItM0</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="34" -->
<!-- PageBreak -->


## Making A Claim Group Medical- Important FAQ

What are network hospitals? What should I do when I reach the hospital (NETWORK)?

. These are hospitals where TPA has a tie up for the cashless hospitalization. There are two kinds of network hospitals; PPN Network
hospitals where cashless services can be obtained for emergency and planned treatments and Standard (Non PPN) network
hospitals where cashless services can be obtained for planned Hospitalization.

. Once you have reached there, please show your ID card for identification. TPA will also send $a$ letter of credit (on pre-authorization)
to the hospital to make sure that they extend credit facility. Please complete the pre-authorization procedure listed earlier. If the
pre-authorization is not done, you must collect all reports and discharge card when you get discharged. Please make sure that you
sign the hospital bill before leaving the hospital. You can then submit the claim along with all the necessary supporting documents
to TPA as a reimbursement . If, however, you go to a non network hospital , it is still advisable to fill the preauthorization form ( use
the copy attached with the Benefits Manual). Please fill the claim form, attach the relevant documents and send it to TPA office for
reimbursement.


### How can I claim my pre & post hospitalization expenses?

. The policy covers pre-hospitalization expenses made prior to 60 days of hospitalization and incurred towards the same illness/
disease due to which hospitalization happens. It also covers all medical expenses for up to 60 days post discharge as advised by the
Medical Practitioner. All bills with summary have to be sent to TPA for reimbursement within 7 days of incurring the expense.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="35" -->
<!-- PageBreak -->


## Making A Claim(1/2) Things To Remember

Always aim to pre-authorize your benefits with the TPA
This will help you in the following ways:

. You will be informed in advance regarding your coverage for the treatment and whether it is covered under your medical plan or
not . This will help you know in advance if your claim may get rejected at a later stage and you do not end up paying out of pocket.

. It will help you ensure that the treatment cost is appropriate and not inflated. as the TPA will be able to cross check costs with the
hospital in question. This will also help TPA in planning your hospitalization expenditure such that you do not run out of the cover
that you are entitled to.

. It will help TPA in registering the impending claim with the insurer

Ensure your dependent list is always updated and claims submitted as per protocols

· Please ensure that all your dependents are covered and have a valid card at the outset itself as it will not be possible to add
dependents at a later stage. Submit your reimbursement claims within timelines from the hospital. Please do not postpone this till
later as it may mean that your claim gets rejected due to late submission .

. Please check that your documents are submitted completely at the first instance itself and originals are submitted wherever
requested for . Do note that incomplete submissions will not be considered as exceptions by the insurers and will only delay the
process further for you and a delay may lead to the claim getting closed. Please also retain a copy of all claim documents submitted
to the insurer

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="36" -->
<!-- PageBreak -->


## Making A Claim (2/2) Things To Remember

Know that it is possible that benefits under your plan could be reduced v/s your eligible sum insured

The following are some common reasons for rejection although these are NOT the only reasons why a claim could be reduced

· Limits for the specific ailment exceed the reasonable cap on ailments listed in the manual,

· Claim amount exceeds the permissible limit under the policy for you ( denied to the extent of the excess),

. Some expense items are non payable for e.g. toiletries , food charges for visitors etc.

Know that it is possible that your claim could also be completely rejected under the plan?

The following are some common reasons for rejection although these are NOT the only reasons why a claim could be rejected

· Treatment taken after leaving the organization. (If you have been transferred from one group business to another, please confirm
with your HR that you have been included for coverage under your new entity)

· Treatment that should have been taken on an outpatient basis (unnecessary inpatient admission and / or no active line of
treatment.) or where hospitalization has been done primarily from a preventive perspective. Please remember that on occasion your
personal doctor may recommend hospital admission for observation purposes however such admissions are not covered under
your medical plan

· Treatment taken is not covered as per policy conditions or excluded, under the policy. Please go through the list of standard
exclusions listed earlier. (for e.g. : Ailment is a because of alcohol abuse is a standard exclusion, similarly cosmetic treatments or
treatments for external conditions like squint correction etc are not covered) . Hospitalization taken in a hospital which is not
covered as per policy conditions (Ex. less than 10 bed hospitals), Admission is before/after the policy period or details of the
member are not updated on the insurer's list of covered members . Additionally in case original documents are not submitted as
per the claim submission protocol.


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="37" -->
<!-- PageBreak -->


## Group Personal Accident Policy

The Group Personal Accident policy covers
expenses for the insured persons (only employee
covered) on account of permanent / temporary, total
or partial disability or death due to an accident.

Accidental Permanent Disablement means
disablement caused due to an accident which
entirely prevents an insured person from attending
to any business or occupation of any and every kind
and which lasts 12 months and at the expiry of that
period is beyond hope of improvement.

Accidental Temporary Total Disablement means
disablement caused due to an accident which
temporarily and totally prevents the Insured Person
from attending to the duties of his usual business or
occupation and shall be payable during such
disablement from the date on which the Insured
person first became disabled.

Accidental Permanent Partial Disablement is a
doctor certified total and continuous loss or
impairment of a body part or sensory organ caused
due to an accident, to the extent specified in the
chart provided by the insurer


<figure>

<!-- PageFooter="AON" -->

</figure>


<figure>

Nadada

Jakuna Man

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<td colspan="2">Plan Name : Group Accident Plan</td>
</tr>
<tr>
<td>Policy Holder</td>
<td>Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Period of the Cover</td>
<td>12 months</td>
</tr>
<tr>
<td>Policy Effective Date</td>
<td>1st Jul 2023</td>
</tr>
<tr>
<td>Expiry Date</td>
<td>30th Jun 2024</td>
</tr>
<tr>
<td>Insurer</td>
<td>Aditya Birla Health Insurance Co. Ltd</td>
</tr>
<tr>
<td>Sum Insured Limit</td>
<td>3 times of Annual Base Salary</td>
</tr>
<tr>
<td>Members covered under the plan</td>
<td>Employees of Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
</table>


<table>
<tr>
<td>Key Benefit Features</td>
<td></td>
</tr>
<tr>
<td>Accidental Death</td>
<td>Covered up to the full sum insured limit</td>
</tr>
<tr>
<td>Permanent Total Disablement</td>
<td>Covered up to the % of sum insured limit</td>
</tr>
<tr>
<td>Permanent Partial Disablement</td>
<td>Covered up to a specified percentage of the full sum insured limit</td>
</tr>
<tr>
<td>Temporary Total Disablement (Weekly Benefit)</td>
<td>Covered for 1% of Sum Insured or INR 5,000 or Actual subject to a maximum limit per week, up to 104 weeks</td>
</tr>
</table>


<figure>

AON

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="39" -->
<!-- PageBreak -->


## Your Plan Details


<table>
<tr>
<td>Key Benefit Features</td>
<td></td>
</tr>
<tr>
<td>Medical Expenses</td>
<td>Limit of 10% of maximum eligible limit or 40% of admissible claims amount or actual whichever is lower subject to a maximum of INR 500,000.</td>
</tr>
<tr>
<td>Education Fund</td>
<td>In case of death of the employee, an Educational Fund for dependent children up to INR 100,000/- for 2 children up to the age of 23 years or 4 years whichever is earlier.</td>
</tr>
<tr>
<td>Ambulance Charges</td>
<td>Covered up to INR 5,000</td>
</tr>
<tr>
<td>Repatriation of Mortal Remains</td>
<td>Covered up to INR 5,000</td>
</tr>
<tr>
<td>Carriage of Dead Body</td>
<td>In the event of death of the insured person due to accident, company shall pay for transportation of insured person's death body to the place resident of a lump sum of 2% of capital sum insured or INR 2,500/- whichever is less.</td>
</tr>
<tr>
<td>Other Coverages</td>
<td>a) Terrorism is covered b) Snake or Animal bite is coved as an accident</td>
</tr>
</table>


AON

<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="40" -->
<!-- PageBreak -->


## Your Plan Details


### Key Exclusions

· Suicide, attempted suicide (whether sane or insane) or intentionally self-inflicted Injury or illness, or sexually transmitted
conditions, mental or nervous disorder, anxiety, stress or depression, Acquired Immune Deficiency Syndrome (AIDS), Human
Immune-deficiency Virus (HIV) infection; or

· Being under the influence of drugs, alcohol, or other intoxicants or hallucinogens unless properly prescribed by a Physician and
taken as prescribed; or

· Participation in an actual or attempted felony, riot, crime, misdemeanor, (excluding traffic violations) or civil commotion; or

· Operating or learning to operate any aircraft or performing duties as a member of the crew on any aircraft; or Scheduled
Aircraft .; or

· Self exposure to needless peril (except in an attempt to save human life);

· Loss due to childbirth or pregnancy

· Self exposure to adventurous sports like sky diving / scuba diving / rock climbing, car racing, etc.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="41" -->
<!-- PageBreak -->


## Making A Claim Group Personal Accident - Claim Procedure


<figure>


The process begins when an employee or beneficiary notifies the HR department, who then informs the insurer and submits the required claims documents within 14 days of the event. Once all relevant documents are obtained, the insurer starts processing the claims. The next step involves claim investigation and review after all required documents have been submitted. A decision point follows, where it is determined whether the claim is approved. If the claim is approved, a cheque is sent to the HR department or NEFT details are shared with HR, who then communicates the information to the employee or beneficiary. If the claim is rejected, the insurer provides a valid reason for the rejection to the HR, employee, or beneficiary.

This process map outlines the sequential steps, decision points, and responsibilities involved in the insurance claim process, ensuring clarity on document submission timelines, claim processing, investigation, approval, and communication of outcomes.
Employee / Beneficiary
notifies HR, who in turn would
intimate Insurer and submit required
claims documents within 14 days of
the event

On obtaining all relevant documents,
Insurer begins processing the claims

Claim Investigation and Review
post submission of all the
required documents

Yes

Is claim
approved
?

No

On approval, the cheque is sent to the HR
or NEFT details shared with HR, from
where the information is shared to the
Employee / Beneficiary

On rejection of the claim, Insurer would
provide a valid reason for the rejection to
HR / Employee / Beneficiary

</figure>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="42" -->
<!-- PageBreak -->


## Making A Claim Document Check List


<table>
<tr>
<td>Sr No.</td>
<td>Document Details for Accidental Death</td>
</tr>
<tr>
<td>1</td>
<td>Completed Claim form with photocopy of ID card</td>
</tr>
<tr>
<td>2</td>
<td>Copy of postmortem examination report</td>
</tr>
<tr>
<td>3</td>
<td>Copy of the first information report from police department / copy of the medico-legal certificate</td>
</tr>
<tr>
<td>4</td>
<td>Original death summary from the hospital</td>
</tr>
<tr>
<td>5</td>
<td>Copy of the legal heir certificate, if the claim is for the death of the principle insured</td>
</tr>
<tr>
<td>6</td>
<td>Viscera Report for death due to poisoning OR snake bite</td>
</tr>
<tr>
<td>7</td>
<td>Death certificate from Municipal Authority OR Gram Panchayat (Applicable for claim reported from rural areas)</td>
</tr>
<tr>
<td>8</td>
<td>Salary slip prior to Accident</td>
</tr>
</table>


<table>
<tr>
<td>Sr No.</td>
<td>Document Details for Disablement Claims</td>
</tr>
<tr>
<td>1</td>
<td>Completed Claim form with photocopy of ID card</td>
</tr>
<tr>
<td>2</td>
<td>Original detailed discharge summary / day care summary from the hospital</td>
</tr>
<tr>
<td>3</td>
<td>Treating doctor's certificate giving details of injuries (How, when and where injury sustained)</td>
</tr>
<tr>
<td>4</td>
<td>Copy of the first information report from police department / copy of the medico-legal certificate</td>
</tr>
<tr>
<td>5</td>
<td>First consultation letter and subsequent treatment papers</td>
</tr>
<tr>
<td>6</td>
<td>Disability certificate from a concerned specialist affiliated with government hospital confirming the extent and nature of disability</td>
</tr>
</table>


This is an indicative list of documents and there may be additional documents required by the insurer.
It is mandatory to provide the details for nomination of beneficiary.


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="43" -->
<!-- PageBreak -->


## Making A Claim Document Check List


<table>
<tr>
<td>Sr No.</td>
<td>Document Details for Temporary Total Disablement</td>
</tr>
<tr>
<td>1</td>
<td>Completed Claim form with photocopy of ID card</td>
</tr>
<tr>
<td>2</td>
<td>Salary Slip prior to Accident</td>
</tr>
<tr>
<td>3</td>
<td>Fitness Certificate by the treating doctor</td>
</tr>
<tr>
<td>4</td>
<td>Medical/Hospital Documents if any</td>
</tr>
</table>


This is an indicative list of documents and there may be
additional documents required by the insurer.
It is mandatory to provide the details for nomination of
beneficiary.


<table>
<tr>
<td>Sr No.</td>
<td>Document Details for Emergency Medical Expenses/Emergency Expenses (Accident Only)</td>
</tr>
<tr>
<td>1</td>
<td>Completed Claim form with photocopy of ID card</td>
</tr>
<tr>
<td>2</td>
<td>Original consolidated hospital bill with breakup of each Item, duly signed by the insured</td>
</tr>
<tr>
<td>3</td>
<td>Original payment receipt of the hospital bill</td>
</tr>
<tr>
<td>4</td>
<td>Original bills, original payment receipts and reports for investigation</td>
</tr>
<tr>
<td>5</td>
<td>Original medicine bills and receipts with corresponding prescriptions</td>
</tr>
<tr>
<td>6</td>
<td>Original invoice/bills for implants (viz. Stent /PHS Mesh / IOL etc.) with original payment Receipts</td>
</tr>
<tr>
<td>7</td>
<td>Treating doctor's certificate giving details of injuries (How, when and where injury sustained) including whether claimant was under the influence of any intoxicating material</td>
</tr>
<tr>
<td>8</td>
<td>Copy of the medico-legal certificate</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="44" -->
<!-- PageBreak -->


## Group Term Life Benefits

Group Term Life Insurance Scheme is
meant to provide life insurance protection
to the employees. The Policy provides for
payment of a lump sum to the nominated
beneficiary in the unfortunate event of the
employee's death due to any cause. Plans
are subject to a Free Cover Limit and
requirement for medical tests. Employees
whose sum insured is above the Free
Cover Limit, will have to undergo medical
test as informed/prescribed by the
insurer. In case any employee fails to
complete the criteria of the insurer, sum
insured benefit will be restricted up to
Free Cover Limit only.

<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee \ Benefits | Manual 2023-24" -->
<!-- PageBreak -->


## Your Plan Details


### Plan Name : Group Term Life


<table>
<tr>
<td>Policy Holder</td>
<td>Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Period of the Cover</td>
<td>12 months</td>
</tr>
<tr>
<td>Policy Effective Date</td>
<td>1st July 2023</td>
</tr>
<tr>
<td>Expiry Date</td>
<td>30th June 2024</td>
</tr>
<tr>
<td>Insurer</td>
<td>HDFC Life Insurance Co. Ltd</td>
</tr>
<tr>
<td>Basis of Sum Assured</td>
<td>3 times of Annual Base Salary (Minimum Sum assured of INR 30 lacs) upto FCL limit of INR 4 Crores - Maximum Age 59 years</td>
</tr>
<tr>
<td>Terminal Illness Rider</td>
<td>100% of Base Life Cover subject to maximum of INR 2 Cr.</td>
</tr>
<tr>
<td>Members covered under the plan</td>
<td>Employees of Novartis Healthcare Pvt Ltd, Novartis India Limited and Sandoz Pvt Ltd.</td>
</tr>
<tr>
<td>Geographical Limits</td>
<td>24*7 World-Wide</td>
</tr>
</table>


<table>
<tr>
<th>Benefit</th>
<th>Coverage</th>
</tr>
<tr>
<td>Death</td>
<td>Due to any cause</td>
</tr>
</table>


<!-- PageFooter="AON" -->
<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="46" -->
<!-- PageBreak -->


## Document Checklist


<table>
<tr>
<th>Type of Claim</th>
<th>Requirement</th>
</tr>
<tr>
<td rowspan="9">Death (all causes of death #)</td>
<td>· Claim Forms Part I: Application Form for Death Claim (Claimant's Statement) # Part II: Physician's Statement, relevant Hospital records and report from the concerned medical specialist giving nature of disability and illness (for Critical Illness claims).</td>
</tr>
<tr>
<td>· Death Certificate issued by a local government body like Municipal Corporation/Village Panchayat</td>
</tr>
<tr>
<td>· Medical Cause of Death Certificate issued by attending physician/hospital</td>
</tr>
<tr>
<td>· Attested True Copies of Indoor case Papers of the hospital(s)</td>
</tr>
<tr>
<td>· Post-mortem Report (Autopsy Report) &amp; Chemical Viscera Report - if performed</td>
</tr>
<tr>
<td>· The Beneficiary : o Photo ID with DOB with relationship to the insured</td>
</tr>
<tr>
<td>o Proof of legal title to the claim proceeds (e.g., legal succession papers, assignment deed etc.)</td>
</tr>
<tr>
<td>· Employer's Certificate</td>
</tr>
<tr>
<td>· Leave Records for the past 3 years</td>
</tr>
<tr>
<td rowspan="3">If Death due to Accident (submit in addition to the above)</td>
<td>All Police Reports / First Information &amp; Final Investigation Report</td>
</tr>
<tr>
<td>Proof of Accident - Panchnama / Inquest Report</td>
</tr>
<tr>
<td>Newspaper cutting / Photographs of the accident - if available</td>
</tr>
</table>


The above is an indicative list of documents, and the insurer reserves the right to ask for additional proofs & documents in support of the
claim. Policyholder shall inform the insurance company of any claim within 30 days of the claim event.


<figure>

<!-- PageFooter="AON" -->

</figure>


<!-- PageFooter="Novartis Group-Employee Benefits Manual 2023-24" -->
<!-- PageNumber="47" -->
<!-- PageBreak -->


## Thank You

About Aon: Aon plc (NYSE:AON) is a leading global
professional services firm providing a broad range of risk,
retirement and health solutions. Our 50,000 colleagues
in 120 countries empower results for clients by using
proprietary data and analytics to deliver insights that
reduce volatility and improve performance. The
information contained herein, and the statements
expressed are of a general nature and are not intended to
address the circumstances of any particular individual or
entity. Although we endeavour to provide accurate and
timely information and use sources, we consider reliable,
there can be no guarantee that such information is
accurate as of the date it is received or that it will
continue to be accurate in the future. No one should act
on such information without appropriate professional
advice after a thorough examination of the particular
situation.

www.aon.com

AON

<!-- PageNumber="48" -->
